## Co-factor swap

#### Co-factors:

NAD+ / NADH: Nicotinamide adenine dinucleotide (NAD+) is reduced to NADH in the conversion of isoamyl aldehyde to isoamyl alcohol. This is an oxidation-reduction reaction where NAD+ acts as an oxidizing agent.

CoASH / Acetyl-CoA: Coenzyme A (CoASH) is used in the formation of acetyl-CoA, which is a substrate for the reaction that synthesizes isoamyl acetate. Acetyl-CoA is a key molecule in metabolic processes and is formed from acetate and CoASH with the input of energy from ATP.

ATP / AMP + PPi: Adenosine triphosphate (ATP) is converted to adenosine monophosphate (AMP) and inorganic pyrophosphate (PPi) in the reaction that activates acetate to acetyl adenylate as a precursor to acetyl-CoA formation.

In the code the following was done:
- Model yeast8 was loaded
- The objective was set to maximize the production of isoamyl acetate
- The theoretical yield was set to production of isoamyl acetate divided by the uptake of glucose
- The algorithm was run and the result was visualized

In [20]:
pip install cobra

Note: you may need to restart the kernel to use updated packages.


In [3]:
# fix the cameo import errors
from collections.abc import Mapping
import os
import pandas as pd
import matplotlib.pyplot as plt
from cameo.core import manipulation
from cameo import fba, flux_variability_analysis
from cobra.io import read_sbml_model
from cameo import load_model
from matplotlib import pyplot as plt
from cameo.flux_analysis.analysis import flux_variability_analysis as fva
from cameo.util import TimeMachine
from cobra.core.gene import GPR
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield
# from cameo.strain_design.heuristic.evolutionary_based import CofactorSwapOptimization

In [8]:
from cameo.strain_design.heuristic.evolutionary_based import CofactorSwapOptimization

ImportError: cannot import name 'find_gene_knockout_reactions' from 'cobra.manipulation.delete' (c:\Users\pauli\anaconda3\Lib\site-packages\cobra\manipulation\delete.py)

In [8]:
model = read_sbml_model("Models/yeast8.xml")

In [10]:
# Choose the reaction that produces isoamyl acetate as the objective.
objective_reaction = model.reactions.r_1180
model.objective = objective_reaction
print(f'Objective set to maximize {objective_reaction.id}: {objective_reaction.reaction}')

Objective set to maximize r_1180: s_0927 --> s_0928


In [18]:
Fluxes=model.optimize()
model.objective = model.reactions.r_1180
py = product_yield(model.reactions.r_1180, model.reactions.r_1714)
swap_optimization = CofactorSwapOptimization(model=model, cofactor_id_swaps=(["NADP_c","NADPH_c"]),objective_function=py)
solution=swap_optimization.run(max_size=10)

NameError: name 'CofactorSwapOptimization' is not defined